# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb.)

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - C (hand-written)
    - C (hand-written with -ffast-math)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
    - Julia (built-in)
    - Julia (hand-written)
    - Julia (hand-written with SIMD)
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [3]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Vector{Float64}:
 0.4317568043478941
 0.8566700130948739
 0.9406424896027232
 0.36100325801111155
 0.9101263273874078
 0.07114436142540148
 0.32748490806439423
 0.32440485955489406
 0.28462320535763386
 0.8187242345811767
 0.09309836351456391
 0.016853186912473728
 0.15508663417261825
 ⋮
 0.4085984931193385
 0.2111478418116366
 0.6452066713042888
 0.33567792448352396
 0.6789094653687731
 0.02194182362210806
 0.9572930967415654
 0.41009686799635103
 0.33173891369319586
 0.654512724495752
 0.7612263318280024
 0.36612761261671256

In [4]:
sum(a)

5.000566333321083e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

In [5]:
@time sum(a)

  0.011696 seconds (1 allocation: 16 bytes)


5.000566333321083e6

In [6]:
@time sum(a)

  0.004769 seconds (1 allocation: 16 bytes)


5.000566333321083e6

In [7]:
@time sum(a)

  0.004699 seconds (1 allocation: 16 bytes)


5.000566333321083e6

The `@time` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a `BenchmarkTools.jl` package to make benchmarking easy and accurate:

In [8]:
 using Pkg
Pkg.add("BenchmarkTools")

In [9]:
using BenchmarkTools  

#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [10]:
using Libdl
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

LoadError: IOError: could not spawn `gcc -fPIC -O3 -msse3 -xc -shared -o 'C:\Users\anton\AppData\Local\Temp\jl_Dnf5JhNdZs.dll' -`: no such file or directory (ENOENT)

In [11]:
c_sum(a)

LoadError: UndefVarError: c_sum not defined

In [12]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

LoadError: UndefVarError: c_sum not defined

In [13]:
c_sum(a) - sum(a)  

LoadError: UndefVarError: c_sum not defined

In [14]:
≈  # alias for the `isapprox` function

isapprox (generic function with 9 methods)

In [15]:
?isapprox

search: isapprox



```
isapprox(x, y; atol::Real=0, rtol::Real=atol>0 ? 0 : √eps, nans::Bool=false[, norm::Function])
```

Inexact equality comparison: `true` if `norm(x-y) <= max(atol, rtol*max(norm(x), norm(y)))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, if an `atol > 0` is not specified, `rtol` defaults to the square root of [`eps`](@ref) of the type of `x` or `y`, whichever is bigger (least precise). This corresponds to requiring equality of about half of the significand digits. Otherwise, e.g. for integer arguments or if an `atol > 0` is supplied, `rtol` defaults to zero.

The `norm` keyword defaults to `abs` for numeric `(x,y)` and to `LinearAlgebra.norm` for arrays (where an alternative `norm` choice is sometimes useful). When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

Note that `x ≈ 0` (i.e., comparing to zero with the default tolerances) is equivalent to `x == 0` since the default `atol` is `0`.  In such cases, you should either supply an appropriate `atol` (or use `norm(x) ≤ atol`) or rearrange your code (e.g. use `x ≈ y` rather than `x - y ≈ 0`).   It is not possible to pick a nonzero `atol` automatically because it depends on the overall scaling (the "units") of your problem: for example, in `x - y ≈ 0`, `atol=1e-9` is an absurdly small tolerance if `x` is the [radius of the Earth](https://en.wikipedia.org/wiki/Earth_radius) in meters, but an absurdly large tolerance if `x` is the [radius of a Hydrogen atom](https://en.wikipedia.org/wiki/Bohr_radius) in meters.

!!! compat "Julia 1.6"
    Passing the `norm` keyword argument when comparing numeric (non-array) arguments requires Julia 1.6 or later.


# Examples

```jldoctest
julia> 0.1 ≈ (0.1 - 1e-10)
true

julia> isapprox(10, 11; atol = 2)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0])
true

julia> 1e-10 ≈ 0
false

julia> isapprox(1e-10, 0, atol=1e-8)
true
```

---

```
isapprox(x; kwargs...) / ≈(x; kwargs...)
```

Create a function that compares its argument to `x` using `≈`, i.e. a function equivalent to `y -> y ≈ x`.

The keyword arguments supported here are the same as those in the 2-argument `isapprox`.


We can now benchmark the C code directly from Julia:

In [16]:
c_bench = @benchmark c_sum($a)

LoadError: UndefVarError: c_sum not defined

In [17]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

LoadError: UndefVarError: c_bench not defined

In [24]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

LoadError: UndefVarError: c_bench not defined

In [ ]:
using Plots
gr()

In [ ]:
using Statistics # bring in statistical support for standard deviations
t = c_bench.times / 1e6 # times in milliseconds
m, σ = minimum(t), std(t)

histogram(t, bins=500,
    xlim=(m - 0.01, m + σ),
    xlabel="milliseconds", ylabel="count", label="")

# 2. C with -ffast-math

If we allow C to re-arrange the floating point operations, then it'll vectorize with SIMD (single instruction, multiple data) instructions.

In [ ]:
const Clib_fastmath = tempname()   # make a temporary file

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = ccall(("c_sum", Clib_fastmath), Float64, (Csize_t, Ptr{Float64}), length(X), X)

In [ ]:
c_fastmath_bench = @benchmark $c_sum_fastmath($a)

In [ ]:
d["C -ffast-math"] = minimum(c_fastmath_bench.times) / 1e6  # in milliseconds

# 3. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [18]:
using Pkg; Pkg.add("PyCall")
using PyCall

In [19]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [20]:
pysum(a)

5.000566333320858e6

In [21]:
pysum(a) ≈ sum(a)

true

In [22]:
py_list_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  6
  --------------
  minimum time:     746.899 ms (0.00% GC)
  median time:      769.583 ms (0.00% GC)
  mean time:        775.178 ms (0.00% GC)
  maximum time:     827.033 ms (0.00% GC)
  --------------
  samples:          7
  evals/sample:     1

In [25]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any, Any} with 1 entry:
  "Python built-in" => 746.899

# 4. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [27]:
using Pkg; Pkg.add("Conda")
using Conda

In [28]:
Conda.add("numpy")

┌ Info: Running `conda install -y numpy` in root environment
└ @ Conda C:\Users\anton\.julia\packages\Conda\sNGum\src\Conda.jl:128


Solving environment: ...working... done

# All requested packages already installed.



In [29]:
numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  6
  --------------
  minimum time:     11.914 ms (0.00% GC)
  median time:      13.624 ms (0.00% GC)
  mean time:        14.135 ms (0.00% GC)
  maximum time:     19.970 ms (0.00% GC)
  --------------
  samples:          353
  evals/sample:     1

In [30]:
numpy_sum(a)

5.000566333321071e6

In [31]:
numpy_sum(a) ≈ sum(a)

true

In [32]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any, Any} with 2 entries:
  "Python numpy"    => 11.9137
  "Python built-in" => 746.899

# 5. Python, hand-written 

In [33]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x000000000E064A60>

In [34]:
py_hand = @benchmark $sum_py($a)

BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  6
  --------------
  minimum time:     938.084 ms (0.00% GC)
  median time:      964.783 ms (0.00% GC)
  mean time:        978.616 ms (0.00% GC)
  maximum time:     1.092 s (0.00% GC)
  --------------
  samples:          6
  evals/sample:     1

In [35]:
sum_py(a)

5.000566333320858e6

In [36]:
sum_py(a) ≈ sum(a)

true

In [37]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any, Any} with 3 entries:
  "Python numpy"        => 11.9137
  "Python hand-written" => 938.084
  "Python built-in"     => 746.899

# 6. Julia (built-in) 

## Written directly in Julia, not in C!

In [38]:
@which sum(a)

sum(a::AbstractArray; dims, kw...) in Base at reducedim.jl:873

In [39]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.190 ms (0.00% GC)
  median time:      4.939 ms (0.00% GC)
  mean time:        5.144 ms (0.00% GC)
  maximum time:     7.963 ms (0.00% GC)
  --------------
  samples:          968
  evals/sample:     1

In [40]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any, Any} with 4 entries:
  "Python numpy"        => 11.9137
  "Python hand-written" => 938.084
  "Python built-in"     => 746.899
  "Julia built-in"      => 4.1896

# 7. Julia (hand-written) 

In [41]:
function mysum(A)   
    s = 0.0 # s = zero(eltype(a))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [42]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     12.268 ms (0.00% GC)
  median time:      13.332 ms (0.00% GC)
  mean time:        13.565 ms (0.00% GC)
  maximum time:     25.836 ms (0.00% GC)
  --------------
  samples:          368
  evals/sample:     1

In [43]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any, Any} with 5 entries:
  "Python numpy"        => 11.9137
  "Julia hand-written"  => 12.2678
  "Python hand-written" => 938.084
  "Python built-in"     => 746.899
  "Julia built-in"      => 4.1896

# 8. Julia (hand-written w. simd) 

In [44]:
function mysum_simd(A)   
    s = 0.0 # s = zero(eltype(A))
    @simd for a in A
        s += a
    end
    s
end

mysum_simd (generic function with 1 method)

In [45]:
j_bench_hand_simd = @benchmark mysum_simd($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.221 ms (0.00% GC)
  median time:      5.157 ms (0.00% GC)
  mean time:        5.392 ms (0.00% GC)
  maximum time:     12.069 ms (0.00% GC)
  --------------
  samples:          923
  evals/sample:     1

In [49]:
mysum_simd(a)

5.000566333321078e6

In [50]:
d["Julia hand-written simd"] = minimum(j_bench_hand_simd.times) / 1e6
d

Dict{Any, Any} with 6 entries:
  "Julia hand-written simd" => 4.2215
  "Python numpy"            => 11.9137
  "Julia hand-written"      => 12.2678
  "Python hand-written"     => 938.084
  "Python built-in"         => 746.899
  "Julia built-in"          => 4.1896

# Summary

In [51]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia built-in..............4.2
Julia hand-written simd.....4.2
Python numpy...............11.9
Julia hand-written.........12.3
Python built-in...........746.9
Python hand-written.......938.1


In [52]:
?rpad

search: rpad macroexpand @macroexpand @macroexpand1 isdirpath tryparse normpath



```
rpad(s, n::Integer, p::Union{AbstractChar,AbstractString}=' ') -> String
```

Stringify `s` and pad the resulting string on the right with `p` to make it `n` characters (code points) long. If `s` is already `n` characters long, an equal string is returned. Pad with spaces by default.

# Examples

```jldoctest
julia> rpad("March", 20)
"March               "
```
